In [1]:
import tensorflow as tf 
import tensorflow.keras as keras
import matplotlib.pyplot as plt

from tensorflow.keras.preprocessing import image
from sklearn.model_selection import train_test_split

# custom
from utils.utils import load_local_dataset_train_test, load_local_dataset, load_local_dataset_tf, preprocess_data_per_tfmodel
from utils.constants import SOURCE_DATASETS, TARGET_DATASETS


In [2]:

TARGET_SIZE = (300, 300)
# BATCH_SIZE = 128
BATCH_SIZE = 8


path = "/Users/felixgerschner/git/ai-prototype/data/target/mechanicalseals_fulllight"
# path = "/Users/felixgerschner/git/ai-prototype/data/source/miniimagenet"
# path = "/Users/felixgerschner/git/ai-prototype/data/source/dagm"
# path = "/Users/felixgerschner/git/ai-prototype/data/source/caltech101"


train = load_local_dataset_tf(path, target_size=TARGET_SIZE, subset="training", batch_size=None)
test = load_local_dataset_tf(path, target_size=TARGET_SIZE, subset="test", batch_size=None)

# X, y = load_local_dataset(path, ".png", target_size=TARGET_SIZE)

print("batches: ", train.cardinality().numpy())

Found 181 files belonging to 2 classes.
Using 145 files for training.
Metal device set to: Apple M1 Pro

systemMemory: 32.00 GB
maxCacheSize: 10.67 GB

Found 181 files belonging to 2 classes.
Using 36 files for validation.
batches:  145


2023-02-05 13:10:31.147286: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-02-05 13:10:31.147598: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [4]:
train_size = train.cardinality().numpy()
test_size = test.cardinality().numpy()

train_size + test_size

181

In [ ]:
import numpy as np

# X = keras.applications.vgg16.preprocess_input(dataset[0])

train_preprocessed = train.take(5)
test_preprocessed = test.take(5)

train_preprocessed = preprocess_data_per_tfmodel(train)
test_preprocessed = preprocess_data_per_tfmodel(test)

classes = train.class_names


# this could also be the output a different Keras model or layer
# input_tensor = keras.layers.Input(shape=(224, 224, 3), name="put it in")

base_model = keras.applications.VGG16(
            weights="imagenet",
            input_shape=(*TARGET_SIZE, 3),
            include_top=False,
        )

for layer in base_model.layers:
    layer.trainable = False

x = base_model.layers[-1].output
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(32, activation="relu")(x)
x = keras.layers.Dense(32, activation="relu")(x)

out = keras.layers.Dense(len(classes), activation="softmax", name="predictions")(x)

model = keras.models.Model(
            inputs=base_model.input, 
            outputs=out)

optimizer = keras.optimizers.RMSprop(learning_rate=1e-4)

model.compile(
    optimizer = optimizer,
    loss = "categorical_crossentropy",
    metrics = ["accuracy"]
)




# images, labels = next(iter(train_preprocessed))

# x_image, y_image = X[0], y[0]


# print(y_image)
# plt.imshow(x_image.astype("uint8"))
# plt.axis("off")

# x_image = np.expand_dims(x_image, axis=0)
# images = np.expand_dims(images, axis=0)

# print(images.shape, " ", labels.shape)

# plt.imshow(images.numpy().astype("uint8"))

# _image = image.img_to_array(images)

# print(_image.shape)

# y_pred = model.predict(images)

# print(labels)
# print(np.argmax(y_pred))

model.summary()

# model.predict(images)

# y_pred = model.predict(images)
# print(y_pred)

# for image, label in preprocessed.take(1):
#     print("Label: ", label.numpy())
#     print("Image shape: ", image.numpy().shape)
#     plt.imshow(image.numpy().astype("uint8"))
#     plt.axis("off")
# import time

# start_time = time.perf_counter()
# for epoch_num in range(2):
#     for sample in dataset:
#         # Performing a training step
#         time.sleep(0.01)
# print("Execution time:", time.perf_counter() - start_time)
    
#     print(type(images[0]))
#     image = keras.applications.vgg16.preprocess_input(images[1])
#     print(type(image))
    


# keras.applications.vgg16.preprocess_input(X_train)


# path = "/Users/felixgerschner/git/ai-prototype/data/source/caltech101"
# load_local_dataset_tf(path, target_size=TARGET_SIZE)


# class_names = data.class_names
# print(class_names)

# for images, labels in data.take(1):
#     print(images.shape, " ", labels.shape)
 
#     print(labels[0])
#     plt.imshow(images[10].numpy().astype("uint8"))
#     plt.axis("off")


# X, y = load_local_dataset(path, img_format, target_size=TARGET_SIZE)


In [ ]:
hist = model.fit(
        train,
        epochs=2,
        validation_data=test,
        verbose=True,
    )

print(hist.history)

In [ ]:
for layer in model.layers:
    print(layer.name, " ", layer.trainable)

In [ ]:
model.layers[-4].name

test_input = test_preprocessed.take(1)

model.predict(test_input)

In [ ]:
from sklearn.metrics import accuracy_score

image, label = next(iter(test_preprocessed))

y_pred = model.predict(np.expand_dims(image[0], axis=0))


pred = np.argmax(y_pred, axis=1)


acc = accuracy_score([1], pred)
print(acc)

print(np.argmax(label, axis=1))
print(pred)
print(image[0].shape)
print(len(image[0].shape))
print(y_pred)
classes

In [ ]:
plt.imshow(image[0].numpy().astype("uint8"))
plt.axis("off")

In [ ]:
y = np.concatenate([y for x, y in test_preprocessed], axis=0)

print(y)

In [ ]:
# data = tf.keras.utils.image_dataset_from_directory(
#   path,
#   validation_split=0.2,
#   subset="training",
#   seed=123,
#   image_size=TARGET_SIZE,
#   batch_size=BATCH_SIZE,
#   interpolation="area"
# )


# for images, labels in data.take(1):
#     print(images.shape, " ", labels.shape)
 
#     print(labels[1])
#     plt.imshow(images[0].numpy().astype("uint8"))
#     plt.axis("off")


In [ ]:
# plt.imshow(X[4].astype("uint8"))
# plt.axis("off")
# X[4].shape

In [ ]:
# from models.tf_models.VGG16_model import VGG16_model


# model = VGG16_model("imagenet", input_shape=(350, 350, 3), 
#                 num_classes=2, build_pre_model=True)

# model.fit_and_save_pre_model(X, y)

In [ ]:
# from models.tf_models.ResNet101_model import ResNet101_model


# model = ResNet101_model("imagenet", input_shape=(350, 350, 3), 
#                 num_classes=2, build_pre_model=True)

# model.fit_and_save_pre_model(X, y)

In [ ]:
# from models.tf_models.DenseNet121_model import DenseNet121_model


# model = DenseNet121_model("imagenet", input_shape=(350, 350, 3), 
#                 num_classes=2, build_pre_model=True)

# model.fit_and_save_pre_model(X, y)

In [ ]:
# from models.tf_models.MobileNet_model import MobileNet_model


# model = MobileNet_model("mechanicalseal_fulllight", input_shape=(350, 350, 3), 
#                 num_classes=2, build_pre_model=True)

# model.fit_and_save_pre_model(X, y)

In [ ]:
# import matplotlib.pyplot as plt

# plt.imshow(X[2], cmap="binary")
# plt.show()

In [ ]:
# from tensorflow.keras.applications.vgg16 import VGG16

# model = VGG16(weights="imagenet", include_top=True)

# model.summary()

In [ ]:
# import tensorflow as tf 


# from sklearn.model_selection import train_test_split

# # custom
# from utils.utils import load_local_dataset, load_local_dataset_train_test, preload_from_directory_old

# path = "/Users/felixgerschner/git/ai-prototype/data/source/miniimagenet" # CRASHES KERNEL
# path = "/Users/felixgerschner/git/ai-prototype/data/source/caltech101" # Works
# path = "/Users/felixgerschner/git/ai-prototype/data/source/dagm" # Works
# path = "/Users/felixgerschner/git/ai-prototype/data/target/mechanicalseals_fulllight" # Works

# X, y = load_local_dataset(path, ".JPEG", input_shape=None) # CRASHES KERNEL
# X, y = load_local_dataset(path, ".jpg", input_shape=(350, 350))
# X, y = load_local_dataset(path, ".PNG", input_shape=(350, 350))
# X, y = load_local_dataset(path, ".png", input_shape=(350, 350))
# X_train, y_train, X_test, y_test = load_local_dataset_train_test(path, ".png", input_shape=(350, 350))



In [ ]:
# X.shape
# X_train.shape, " ", X_test.shape

In [ ]:
# import matplotlib.pyplot as plt

# plt.imshow(X[2523].astype("uint8"))
# plt.show()
# y[2523]

In [7]:
import pandas as pd


df = pd.DataFrame(columns=["a", "b", "c"], data=[[1, 2, 3]])
print(df)

   a  b  c
0  1  2  3


In [10]:
df.to_numpy()[0].shape

(3,)

In [12]:
liste = []

liste.append(df.to_numpy()[0])
liste.append(df.to_numpy()[0])

print(liste)

[array([1, 2, 3]), array([1, 2, 3])]


In [13]:
df_new = pd.DataFrame(columns=["a", "b", "c"], data=liste)

In [14]:
df_new

,a,b,c
0,1,2,3
1,1,2,3
